## In this notebook we run thought the data aquisitin pipeline to get derive results discussed in Samuel Heczko's masters thesis

In [10]:
import glob
import os
import connectome
import nibabel as nib
import pandas as pd
import regresson

import sys; sys.path
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold



### 1. Get the connectomes from preregistered fmri images
In the thesis we largely conducted analysis on thethe Amsterdam Open MRI Collection (AOMIC). We used special preprocessed files and the data proveided by the AOIMIC people. We derive thea analysis in __ steps. All steps are done to all 900 subjects. We loop following proceedure over all atalases considererd.
1. Calcluate the time series data for each brain region for each atlas (extract_connectomes.py)
2. Compute the connectome of for each subject
3. For each atlas save the upper triangle of the connectivity matrix along the information which brain regions are connected in this connection.


Let's start with setting up the parameters

In [2]:
##defime global parameters
cluster = False
path = 'data/' ##cahnge for cluster
correlation_measure = 'correlation' #can be also tangent, partial
res_n = 1
res = f'{res_n}x{res_n}x{res_n}'
correlation_measure='correlation' ##set for calculation of the brain connectome, choose from 'correlation', 'tangent', 'partial' as implemented by nilearn
#templateICBM = '/kyb/agks/sheczko/Downloads/MastersThesis/code/data/templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii' ##use the ICBM T1 template


##add the data and names of thigs
if cluster:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*.nii') ##load up a subset of the subejct images

else:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*000*.nii') ##load up all the subejct images
subjects_idxs = []
for s_n in imgs_paths:
    subjects_idxs.append(s_n.split('_')[-1].split('.')[0]) #split the path to extract only the number of the subject from path



atlases = glob.glob(path + '/atlases/lawrance2021/label/Human/ICBM/*.nii.gz') ##get the atlases
anatomical_labels = glob.glob(path + '/atlases/lawrance2021/label/Human/Anatomical-labels-csv/*.csv') #get the anatomical labels (where available)
anatomical_label_names = []
for a_l in anatomical_labels:
    anatomical_label_names.append(a_l.split('/')[-1].split('.')[0]) #split the path to extract only the name of the atlas


##loop over atlases

In [4]:
subjects_idxs

['0008', '0005', '0003', '0006', '0002', '0004', '0001', '0007']

Contiune with looping over the list of atlases and saving a csv wiht connectivity for all subejcts.

In [3]:
for atlas_path in atlases: ##loop over atlases
    atlas_name = (atlas_path.split('/')[-1].split('.')[0].split('_')[0]) #split the atlas path so we
    print(atlas_name)
    al_p = (any(n == atlas_name for n in anatomical_label_names)) ##find wheter we have the anatomical labellings for this atlas

    if al_p:
        anatomic_path = path + f'/atlases/lawrance2021/label/Human/Anatomical-labels-csv/' + atlas + '.csv'
        ana_labels = pd.read_csv(anatomic_path,names=colnames, header=None)
    else:
        ana_labels = None


    time_series = connectome.calculate_time_series(atlas_path = atlas_path,imgs_paths=imgs_paths)     ##get the time series of the from all subjects, using the atlas defined

    correlation_matrices, _ =  connectome.connectome(time_series = time_series,correlation_measure='correlation') #get the connectivity matrices

    df_ = connectome.save_connectomes_df(correlation_matrices,anatomical_label_presence = al_p, anatomic_labels = ana_labels,path_to_save = path + 'results/', atlas_name = atlas_name, n_subjects = correlation_matrices.shape[0], correlation_measure = correlation_measure,subject_ixds = subjects_idxs)
    


DS00108
DS06481


KeyboardInterrupt: 

# find the regressons using the features extracted earlier
Now that we have extracted the features of our fMRI data, we are ready to apply the regression models to calcualte the predictability of our target variables. We subdivide this task into three steps:
1. loading and formatting our connectivity data
2. loading and formattin our target variables
3. fitting the desired model on the defined x and y

This process needs to be repeated over all the atlases and other variables we have been looping over. This allows us to asses the question which steps of the pipline architecture have the largest effect on the prediction accuracy.



In [13]:

##let's find the paths to the the csv connectivity data we ahve
data_paths = glob.glob(path + '/results/connectomes/*.csv')
regressors_df =  regresson.load_regressors(path + 'func_images/AOMIC/regressors/*.txt')
GCA = regressors_df.regressor_iq.values
bmi = regressors_df.regressor_bmi.values
##lets define pur target variable
 #concatenate cognitive metrics into single variable
cognition = ['GCA','bmi']
#cognition = ['PMAT Correct', 'PMAT Response Time']
cog_metric = np.transpose(np.asarray([GCA, bmi]))

for data_path in data_paths: ##loop over atlases

    print(data_path.split('/')[-1])


    loaded_data = regresson.load_data(data_path)

    #concatenate cognitive metrics into single variable
    cognition = ['GCA','bmi']
    #cognition = ['PMAT Correct', 'PMAT Response Time']
    cog_metric = np.transpose(np.asarray([GCA, bmi]))
    #cog_metric = np.transpose(np.asarray([pmat_cr, pmat_med]))
    cog_metric



    #set the number of permutations you want to perform
    perm = 5
    #set the number of cross-validation loops you want to perform
    cv_loops = 5
    #set the number of folds you want in the inner and outer folds of the nested cross-validation
    k = 3
    #set the proportion of data you want in your training set
    train_size = .8
    #set the number of variable you want to predict to be the number of variables stored in the cognition variablse
    n_cog = np.size(cognition)

    #set regression model type

    regr = Ridge(fit_intercept = True, max_iter=1000000)

    #set hyperparameter grid space you want to search through for the model
    alphas = np.linspace(100, 1000, num=10, endpoint=True, dtype=None, axis=0)

    #set the param grid be to the hyperparamters you want to search through
    paramGrid ={'alpha': alphas}

    #set x data to be the input variable you want to use
    #ie fc, sc, or hc
    X = fc
    X[X<0] = 0


    # standardization -- optional (test if works better)
    #X_norm = object.fit_transform(X) 
    #print(scale)

    #set y to be the cognitive metrics you want to predict
    Y = cog_metric
    #Y - Y.mean(axis=0)

    #set the number of features 
    n_feat = X.shape[1]


    #create arrays to store variables
    #r^2 - coefficient of determination
    r2 = np.zeros([perm,n_cog])
    #explained variance
    var = np.zeros([perm,n_cog])
    #correlation between true and predicted (aka prediction accuracy)
    corr = np.zeros([perm,n_cog])
    #optimised alpha (hyperparameter)
    opt_alpha = np.zeros([perm,n_cog])
    #predictions made by the model
    preds = np.zeros([perm,n_cog,int(np.ceil(X.shape[0]*(1-train_size)))])
    #true test values for cognition
    cogtest = np.zeros([perm,n_cog,int(np.ceil(X.shape[0]*(1-train_size)))])
    #feature importance extracted from the model
    featimp = np.zeros([perm,n_feat,n_cog])



        #iterate through permutations
    for p in range(perm):
        #print permutation # you're on
        print('Permutation %d' %(p+1))
        #split data into train and test sets
        x_train, x_test, cog_train, cog_test = train_test_split(X, Y, test_size=1-train_size, 
                                                                shuffle=True, random_state=p)

        
        #iterate through the cognitive metrics you want to predict
        for cog in range (n_cog):

            #print cognitive metrics being predicted 
            print ("Cognition: %s" % cognition[cog])
            
            #set y values for train and test based on     
            y_train = cog_train[:,cog]
            y_test = cog_test[:,cog]
            
            #store all the y_test values in a separate variable that can be accessed later if needed
            cogtest[p,cog,:] = y_test


            #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
            nested_scores = []
            best_params = []
            

            #optimise regression model using nested CV
            print('Training Models')
            
            #go through the loops of the cross validation
            for i in range(cv_loops):


                #set parameters for inner and outer loops for CV
                inner_cv = KFold(n_splits=k, shuffle=True, random_state=i)
                outer_cv = KFold(n_splits=k, shuffle=True, random_state=i)
                
                #define regressor with grid-search CV for inner loop
                gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, 
                                        verbose=0, cv=inner_cv, scoring='r2')

                #fit regressor
                gridSearch.fit(x_train, y_train)

                #save parameters corresponding to the best score
                best_params.append(list(gridSearch.best_params_.values()))

                #call cross_val_score for outer loop
                nested_score = cross_val_score(gridSearch, X=x_train, y=y_train, cv=outer_cv, 
                                            scoring='r2', verbose=1)

                #record nested CV scores
                nested_scores.append(np.median(nested_score))

                #print how many cv loops are complete
                print("%d/%d Complete" % (i+1,cv_loops))
                
            #once all CV loops are complete, fit models based on optimised hyperparameters    
            print('Testing Models')


            #save optimised alpha values
            opt_alpha[p,cog] = np.median(best_params)


            #fit model using optimised hyperparameter
            model = Ridge(fit_intercept = True, alpha = opt_alpha[p,cog], max_iter=1000000)
            model.fit(x_train, y_train);
            
            #compute r^2 (coefficient of determination) 
            r2[p,cog]=model.score(x_test,y_test)

            #generate predictions from model
            preds[p,cog,:] = model.predict(x_test).ravel()
            
            #compute explained variance 
            var[p,cog] = explained_variance_score(y_test, preds[p,cog,:])

            #compute correlation between true and predicted
            corr[p,cog] = np.corrcoef(y_test, preds[p,cog,:])[1,0]

            #extract feature importance
            featimp[p,:,cog] = model.coef_

    path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'r2.txt'

    #save all of the outputs once everything is done
    #filepath is where you want to store it

    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'r2.txt', r2, delimiter=',')
    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'var.txt', var, delimiter=',')
    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'corr.txt', corr, delimiter=',')
    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'opt_alpha.txt', opt_alpha, delimiter=',')

    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'iq_ip.txt', featimp[:,:,0], delimiter=',')
    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'bmi_ip.txt', featimp[:,:,1], delimiter=',')


    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'iq_pres.txt', preds[:,0,:], delimiter=',')
    np.savetxt(path + 'results/regression/' + data_path.split('/')[-1].split('.')[0] + 'bmi_preds.txt', preds[:,1,:], delimiter=',')





n_sub-881_correlationType-correlation_atlas-Schaefer1000.csv


In [8]:
regressors.head()

,regressor_neo_n,regressor_ses,regressor_crystal,regressor_neo_o,regressor_fluid,regressor_neo_e,regressor_neo_c,regressor_age,regressor_iq,regressor_bmi,regressor_neo_a,missing,regressor_edu,subj,regressor_memory,regressor_bas_reward,subj_ids
0,28,2.0,33.0,50,77.0,50,47,22.00,159.0,23.0,39,sub-0009,0,proc sub-0001,49.0,16,1.0
1,30,5.5,39.0,47,97.0,46,45,21.75,199.0,20.0,31,sub-0019,0,proc sub-0002,63.0,19,2.0
2,34,3.0,38.0,41,122.0,40,29,25.25,227.0,31.0,45,sub-0020,1,proc sub-0003,67.0,14,3.0
3,29,5.0,52.0,33,149.0,36,44,22.50,270.0,20.0,36,sub-0023,1,proc sub-0004,69.0,18,4.0
4,27,4.5,43.0,35,112.0,44,44,22.25,212.0,23.0,40,sub-0026,1,proc sub-0005,57.0,17,5.0
